In [1]:
import pandas as pd
import numpy as np
from metrics import TCS
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = 'plotly_white'
plot_template = dict(
    layout=go.Layout({
        'font_size': 8,
        'xaxis_title_font_size': 8,
        'yaxis_title_font_size': 8,
        }   
))
hydro_stations = ['Tangnaihai','Guide','Xunhua']
hydrostation_abbrs = {'Tangnaihai':'TNH','Guide':'GD','Xunhua':'XH'}
hydrostation_channel = {'Tangnaihai':'3','Guide':'14','Xunhua':'9'}


In [14]:
pred_years = np.arange(2016,2020)
index = pd.date_range(start=f'{pred_years[0]}-01-01',end=f'{pred_years[-1]}-12-31',freq='MS')
for hydro_station in hydro_stations:
    hydro_station_abbr = hydrostation_abbrs[hydro_station]
    hydro_station_channel = hydrostation_channel[hydro_station]
    pred_df_swatp_simyr = pd.DataFrame(index=index,columns=['obs','pred'])
    pred_df_swatp_simyr.index.name='date'
    pred_df_swatp_ssp126 = pd.DataFrame(index=index,columns=['obs','pred'])
    pred_df_swatp_ssp126.index.name = 'date'
    pred_df_swatp_ssp245 = pd.DataFrame(index=index,columns=['obs','pred'])
    pred_df_swatp_ssp245.index.name = 'date'
    pred_df_swatp_ssp585 = pd.DataFrame(index=index,columns=['obs','pred'])
    pred_df_swatp_ssp585.index.name = 'date'
    pred_df_swatp_arima_simyr = pd.DataFrame(index=index,columns=['obs','pred'])
    pred_df_swatp_arima_simyr.index.name = 'date'
    pred_df_swatp_arima_ssp126 = pd.DataFrame(index=index,columns=['obs','pred'])
    pred_df_swatp_arima_ssp126.index.name = 'date'
    pred_df_swatp_arima_ssp245 = pd.DataFrame(index=index,columns=['obs','pred'])
    pred_df_swatp_arima_ssp245.index.name = 'date'
    pred_df_swatp_arima_ssp585 = pd.DataFrame(index=index,columns=['obs','pred'])
    pred_df_swatp_arima_ssp585.index.name = 'date'
    obs_df = pd.read_csv(f'../data/{hydro_station.lower()}_natural_monthly_flow.csv',index_col=['date'],parse_dates=['date'])
    obs_his = obs_df.copy().loc[:f'{pred_years[0]-1}-12-31']
    obs_his_monthly_mean = obs_his.groupby(obs_his.index.month)['flow(m^3/s)'].mean()
    # Create a new DataFrame with monthly average flow for 2015
    df_monthly_avg = pd.DataFrame(index=index)
    # Replicate the monthly average flow for years 2015-2019
    for year in range(pred_years[0],pred_years[-1]+1):
        for month in range(1,13):   
            df_monthly_avg.loc[f'{year}-{month}-01','flow(m^3/s)'] = obs_his_monthly_mean[month]

    arima_pred = pd.read_csv(f'../result/ARIMAPredData/seasonal_decompose_multiplicative_arima_pred_{hydro_station.lower()}_2015_{pred_years[-1]}.csv',index_col=['date'],parse_dates=['date'])
    SWATPlus_simu = pd.read_csv(f'../result/SWATPlusCalValSimData/Channel_{hydro_station_channel}_Monthly_River-Flow_{hydro_station}_Sim1972_2019.csv',index_col=['Date'],parse_dates=['Date'])
    SWATPlus_simu = SWATPlus_simu.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31',:]
    SWATPlus_simu = SWATPlus_simu.sort_index()
    SWATPlus_simu.index = index
    for pred_year in pred_years:
        date_range = pd.date_range(start=f'{pred_year}-01-01',end=f'{pred_year}-12-31',freq='MS')
        set_index = pd.DatetimeIndex(date_range)
        pred_data_swatp_simy = pd.read_csv(f'../result/SWATPlusPredUsingMeteSimYearData/Channel_{hydro_station_channel}_Monthly_River-Flow_{hydro_station_abbr}_pred{pred_year}.csv',index_col=['Date'],parse_dates=['Date'])
        pred_data_swatp_clip_ssp126 = pd.read_csv(f'../result/SWATPlusPredUsingClimatePatternData/Channel_{hydro_station_channel}_Monthly_River-Flow_{hydro_station_abbr}_ssp126_pred{pred_year}.csv',index_col=['Date'],parse_dates=['Date'])
        pred_data_swatp_clip_ssp245 = pd.read_csv(f'../result/SWATPlusPredUsingClimatePatternData/Channel_{hydro_station_channel}_Monthly_River-Flow_{hydro_station_abbr}_ssp245_pred{pred_year}.csv',index_col=['Date'],parse_dates=['Date'])
        pred_data_swatp_clip_ssp585 = pd.read_csv(f'../result/SWATPlusPredUsingClimatePatternData/Channel_{hydro_station_channel}_Monthly_River-Flow_{hydro_station_abbr}_ssp585_pred{pred_year}.csv',index_col=['Date'],parse_dates=['Date'])  
        pred_df_swatp_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','pred'] = pred_data_swatp_simy.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','Value'].values
        pred_df_swatp_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','obs'] = obs_df.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','flow(m^3/s)'].values
        pred_df_swatp_ssp126.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','pred'] = pred_data_swatp_clip_ssp126.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','Value'].values
        pred_df_swatp_ssp126.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','obs'] = obs_df.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','flow(m^3/s)'].values
        pred_df_swatp_ssp245.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','pred'] = pred_data_swatp_clip_ssp245.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','Value'].values
        pred_df_swatp_ssp245.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','obs'] = obs_df.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','flow(m^3/s)'].values
        pred_df_swatp_ssp585.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','pred'] = pred_data_swatp_clip_ssp585.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','Value'].values
        pred_df_swatp_ssp585.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','obs'] = obs_df.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','flow(m^3/s)'].values
        pred_df_swatp_arima_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','pred'] = pred_data_swatp_simy.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','Value'].values
        pred_df_swatp_arima_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','obs'] = obs_df.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','flow(m^3/s)'].values
        pred_df_swatp_arima_ssp126.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','pred'] = pred_data_swatp_clip_ssp126.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','Value'].values
        pred_df_swatp_arima_ssp126.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','obs'] = obs_df.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','flow(m^3/s)'].values
        pred_df_swatp_arima_ssp245.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','pred'] = pred_data_swatp_clip_ssp245.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','Value'].values
        pred_df_swatp_arima_ssp245.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','obs'] = obs_df.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','flow(m^3/s)'].values  
        pred_df_swatp_arima_ssp585.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','pred'] = pred_data_swatp_clip_ssp585.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','Value'].values
        pred_df_swatp_arima_ssp585.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','obs'] = obs_df.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','flow(m^3/s)'].values
        # Save the corrected predictions for each SSP scenario
        pred_df_swatp_arima_ssp126.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_ARIMA_corrected_pred_obs_ssp126_{pred_years[0]}_{pred_years[-1]}.csv')
        pred_df_swatp_arima_ssp245.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_ARIMA_corrected_pred_obs_ssp245_{pred_years[0]}_{pred_years[-1]}.csv')
        pred_df_swatp_arima_ssp585.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_ARIMA_corrected_pred_obs_ssp585_{pred_years[0]}_{pred_years[-1]}.csv')
        # Save the uncorrected predictions for each SSP scenario
        pred_df_swatp_ssp126.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_pred_obs_ssp126_{pred_years[0]}_{pred_years[-1]}.csv')
        pred_df_swatp_ssp245.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_pred_obs_ssp245_{pred_years[0]}_{pred_years[-1]}.csv')
        pred_df_swatp_ssp585.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_pred_obs_ssp585_{pred_years[0]}_{pred_years[-1]}.csv')
        # arima_pred_5_10 = arima_pred.loc[f'{pred_year}-05-01':f'{pred_year}-11-30']
        arima_pred_6_9 = arima_pred.loc[f'{pred_year}-06-01':f'{pred_year}-10-31']
        arima_pred_6_9_sum = arima_pred_6_9['flow(m^3/s)'].sum()
        arima_pred_1_5 = arima_pred.loc[f'{pred_year}-01-01':f'{pred_year}-05-31']
        arima_pred_1_5_sum = arima_pred_1_5['flow(m^3/s)'].sum()
        arima_pred_11_12 = arima_pred.loc[f'{pred_year}-11-01':f'{pred_year}-12-31']
        arima_pred_11_12_sum = arima_pred_11_12['flow(m^3/s)'].sum()
        arima_pred_other_sum = arima_pred_1_5_sum + arima_pred_11_12_sum
        # print(arima_pred_5_10_sum)
        # pred_data_5_10 = pred_df_swatplus_arima.loc[f'{pred_year}-05-01':f'{pred_year}-11-30']
        swatp_simyr_pred_data_6_9 = pred_df_swatp_arima_simyr.loc[f'{pred_year}-06-01':f'{pred_year}-10-31']
        swatp_simyr_pred_data_6_9_sum = swatp_simyr_pred_data_6_9['pred'].sum()
        swatp_simyr_pred_data_1_5 = pred_df_swatp_arima_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-05-31']
        swatp_simyr_pred_data_1_5_sum = swatp_simyr_pred_data_1_5['pred'].sum()
        swatp_simyr_pred_data_11_12 = pred_df_swatp_arima_simyr.loc[f'{pred_year}-11-01':f'{pred_year}-12-31']
        swatp_simyr_pred_data_11_12_sum = swatp_simyr_pred_data_11_12['pred'].sum()
        swatp_simyr_pred_data_other_sum = swatp_simyr_pred_data_1_5_sum + swatp_simyr_pred_data_11_12_sum
        # print(pred_data_5_10_sum)
        swatp_simyr_factor_6_9 = arima_pred_6_9_sum/swatp_simyr_pred_data_6_9_sum
        # print(arima_pred_6_9_sum,swatp_simyr_pred_data_6_9_sum,swatp_simyr_factor_6_9)
        swatp_simyr_factor_other = arima_pred_other_sum/swatp_simyr_pred_data_other_sum
        # print('SWATPlus SimYr',arima_pred_other_sum,swatp_simyr_pred_data_other_sum,swatp_simyr_factor_other)
        swatp_ssp126_pred_data_6_9 = pred_df_swatp_ssp126.loc[f'{pred_year}-06-01':f'{pred_year}-10-31']
        swatp_ssp126_pred_data_6_9_sum = swatp_ssp126_pred_data_6_9['pred'].sum()
        swatp_ssp126_pred_data_1_5 = pred_df_swatp_ssp126.loc[f'{pred_year}-01-01':f'{pred_year}-05-31']
        swatp_ssp126_pred_data_1_5_sum = swatp_ssp126_pred_data_1_5['pred'].sum()
        swatp_ssp126_pred_data_11_12 = pred_df_swatp_ssp126.loc[f'{pred_year}-11-01':f'{pred_year}-12-31']
        swatp_ssp126_pred_data_11_12_sum = swatp_ssp126_pred_data_11_12['pred'].sum()
        swatp_ssp126_pred_data_other_sum = swatp_ssp126_pred_data_1_5_sum + swatp_ssp126_pred_data_11_12_sum
        swatp_ssp126_factor_6_9 = arima_pred_6_9_sum/swatp_ssp126_pred_data_6_9_sum
        # print(arima_pred_6_9_sum,swatp_ssp126_pred_data_6_9_sum,swatp_ssp126_factor_6_9)
        swatp_ssp126_factor_other = arima_pred_other_sum/swatp_ssp126_pred_data_other_sum
        # print(arima_pred_other_sum,swatp_ssp126_pred_data_other_sum,swatp_ssp126_factor_other)
        swatp_ssp245_pred_data_6_9 = pred_df_swatp_ssp245.loc[f'{pred_year}-06-01':f'{pred_year}-10-31']
        swatp_ssp245_pred_data_6_9_sum = swatp_ssp245_pred_data_6_9['pred'].sum()
        swatp_ssp245_pred_data_1_5 = pred_df_swatp_ssp245.loc[f'{pred_year}-01-01':f'{pred_year}-05-31']    
        swatp_ssp245_pred_data_1_5_sum = swatp_ssp245_pred_data_1_5['pred'].sum()
        swatp_ssp245_pred_data_11_12 = pred_df_swatp_ssp245.loc[f'{pred_year}-11-01':f'{pred_year}-12-31']
        swatp_ssp245_pred_data_11_12_sum = swatp_ssp245_pred_data_11_12['pred'].sum()
        swatp_ssp245_pred_data_other_sum = swatp_ssp245_pred_data_1_5_sum + swatp_ssp245_pred_data_11_12_sum
        swatp_ssp245_factor_6_9 = arima_pred_6_9_sum/swatp_ssp245_pred_data_6_9_sum
        # print(arima_pred_6_9_sum,swatp_ssp245_pred_data_6_9_sum,swatp_ssp245_factor_6_9)
        swatp_ssp245_factor_other = arima_pred_other_sum/swatp_ssp245_pred_data_other_sum
        # print(arima_pred_other_sum,swatp_ssp245_pred_data_other_sum,swatp_ssp245_factor_other)
        swatp_ssp585_pred_data_6_9 = pred_df_swatp_ssp585.loc[f'{pred_year}-06-01':f'{pred_year}-10-31']
        swatp_ssp585_pred_data_6_9_sum   = swatp_ssp585_pred_data_6_9['pred'].sum()
        swatp_ssp585_pred_data_1_5 = pred_df_swatp_ssp585.loc[f'{pred_year}-01-01':f'{pred_year}-05-31']
        swatp_ssp585_pred_data_1_5_sum = swatp_ssp585_pred_data_1_5['pred'].sum()
        swatp_ssp585_pred_data_11_12 = pred_df_swatp_ssp585.loc[f'{pred_year}-11-01':f'{pred_year}-12-31']
        swatp_ssp585_pred_data_11_12_sum = swatp_ssp585_pred_data_11_12['pred'].sum()
        swatp_ssp585_pred_data_other_sum = swatp_ssp585_pred_data_1_5_sum + swatp_ssp585_pred_data_11_12_sum
        swatp_ssp585_factor_6_9 = arima_pred_6_9_sum/swatp_ssp585_pred_data_6_9_sum
        # print(arima_pred_6_9_sum,swatp_ssp585_pred_data_6_9_sum,swatp_ssp585_factor_6_9)
        swatp_ssp585_factor_other = arima_pred_other_sum/swatp_ssp585_pred_data_other_sum
        # print(arima_pred_other_sum,swatp_ssp585_pred_data_other_sum,swatp_ssp585_factor_other)
        pred_df_swatp_arima_simyr.loc[f'{pred_year}-06-01':f'{pred_year}-10-31','pred'] = pred_df_swatp_arima_simyr.loc[f'{pred_year}-06-01':f'{pred_year}-10-31','pred']*swatp_simyr_factor_6_9  
        pred_df_swatp_arima_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-05-31','pred'] = pred_df_swatp_arima_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-05-31','pred']*swatp_simyr_factor_other
        pred_df_swatp_arima_simyr.loc[f'{pred_year}-11-01':f'{pred_year}-12-31','pred'] = pred_df_swatp_arima_simyr.loc[f'{pred_year}-11-01':f'{pred_year}-12-31','pred']*swatp_simyr_factor_other
        # print(pred_df_swatp_arima_simyr)
        pred_df_swatp_arima_ssp126.loc[f'{pred_year}-06-01':f'{pred_year}-10-31','pred'] = pred_df_swatp_arima_ssp126.loc[f'{pred_year}-06-01':f'{pred_year}-10-31','pred']*swatp_ssp126_factor_6_9
        pred_df_swatp_arima_ssp126.loc[f'{pred_year}-01-01':f'{pred_year}-05-31','pred'] = pred_df_swatp_arima_ssp126.loc[f'{pred_year}-01-01':f'{pred_year}-05-31','pred']*swatp_ssp126_factor_other
        pred_df_swatp_arima_ssp126.loc[f'{pred_year}-11-01':f'{pred_year}-12-31','pred'] = pred_df_swatp_arima_ssp126.loc[f'{pred_year}-11-01':f'{pred_year}-12-31','pred']*swatp_ssp126_factor_other
        pred_df_swatp_arima_ssp245.loc[f'{pred_year}-06-01':f'{pred_year}-10-31','pred'] = pred_df_swatp_arima_ssp245.loc[f'{pred_year}-06-01':f'{pred_year}-10-31','pred']*swatp_ssp245_factor_6_9
        pred_df_swatp_arima_ssp245.loc[f'{pred_year}-01-01':f'{pred_year}-05-31','pred'] = pred_df_swatp_arima_ssp245.loc[f'{pred_year}-01-01':f'{pred_year}-05-31','pred']*swatp_ssp245_factor_other
        pred_df_swatp_arima_ssp245.loc[f'{pred_year}-11-01':f'{pred_year}-12-31','pred'] = pred_df_swatp_arima_ssp245.loc[f'{pred_year}-11-01':f'{pred_year}-12-31','pred']*swatp_ssp245_factor_other
        pred_df_swatp_arima_ssp585.loc[f'{pred_year}-06-01':f'{pred_year}-10-31','pred'] = pred_df_swatp_arima_ssp585.loc[f'{pred_year}-06-01':f'{pred_year}-10-31','pred']*swatp_ssp585_factor_6_9
        pred_df_swatp_arima_ssp585.loc[f'{pred_year}-01-01':f'{pred_year}-05-31','pred'] = pred_df_swatp_arima_ssp585.loc[f'{pred_year}-01-01':f'{pred_year}-05-31','pred']*swatp_ssp585_factor_other
        pred_df_swatp_arima_ssp585.loc[f'{pred_year}-11-01':f'{pred_year}-12-31','pred'] = pred_df_swatp_arima_ssp585.loc[f'{pred_year}-11-01':f'{pred_year}-12-31','pred']*swatp_ssp585_factor_other
    pred_df_swatp_arima_simyr.to_csv(f'../result/SWATPlusPredUsingMeteSimYearData/{hydro_station}_SWATPlus_corrected_pred_obs_{pred_years[0]}_{pred_years[-1]}.csv')
    pred_df_swatp_simyr.to_csv(f'../result/SWATPlusPredUsingMeteSimYearData/{hydro_station}_SWATPlus_pred_obs_{pred_years[0]}_{pred_years[-1]}.csv')
    pred_df_swatp_ssp126.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_pred_obs_ssp126_{pred_years[0]}_{pred_years[-1]}.csv')
    pred_df_swatp_ssp245.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_pred_obs_ssp245_{pred_years[0]}_{pred_years[-1]}.csv')
    pred_df_swatp_ssp585.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_pred_obs_ssp585_{pred_years[0]}_{pred_years[-1]}.csv')
    pred_df_swatp_arima_ssp126.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_corrected_pred_obs_ssp126_{pred_years[0]}_{pred_years[-1]}.csv')
    pred_df_swatp_arima_ssp245.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_corrected_pred_obs_ssp245_{pred_years[0]}_{pred_years[-1]}.csv')
    pred_df_swatp_arima_ssp585.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_corrected_pred_obs_ssp585_{pred_years[0]}_{pred_years[-1]}.csv')
    print('R2,TSC for average monthly flow:',r2_score(obs_df.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)'],df_monthly_avg['flow(m^3/s)']),TCS(obs_df.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)'],df_monthly_avg['flow(m^3/s)']))
    print('R2,TCS for corrected SWATPlus-ARIMA:',r2_score(pred_df_swatp_arima_simyr['obs'],pred_df_swatp_arima_simyr['pred']),TCS(pred_df_swatp_arima_simyr['obs'],pred_df_swatp_arima_simyr['pred']))
    print('R2,TCS for SARIMA:',
    r2_score(obs_df.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)'],arima_pred.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)']),
    TCS(obs_df.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)'],arima_pred.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)']),)
    print('R2,TCS for SWATPlus SimYr:',r2_score(pred_df_swatp_simyr['obs'],pred_df_swatp_simyr['pred']),TCS(pred_df_swatp_simyr['obs'],pred_df_swatp_simyr['pred']))
    print('R2,TCS for SWATPlus SSP126:',r2_score(pred_df_swatp_ssp126['obs'],pred_df_swatp_ssp126['pred']),TCS(pred_df_swatp_ssp126['obs'],pred_df_swatp_ssp126['pred']))
    print('R2,TCS for SWATPlus SSP245:',r2_score(pred_df_swatp_ssp245['obs'],pred_df_swatp_ssp245['pred']),TCS(pred_df_swatp_ssp245['obs'],pred_df_swatp_ssp245['pred']))
    print('R2,TCS for SWATPlus SSP585:',r2_score(pred_df_swatp_ssp585['obs'],pred_df_swatp_ssp585['pred']),TCS(pred_df_swatp_ssp585['obs'],pred_df_swatp_ssp585['pred']))
    print('R2,TCS for SWATPlus+ARIMA SimYr:',r2_score(pred_df_swatp_arima_simyr['obs'],pred_df_swatp_arima_simyr['pred']),TCS(pred_df_swatp_arima_simyr['obs'],pred_df_swatp_arima_simyr['pred']))
    print('R2,TCS for SWATPlus+ARIMA SSP126:',r2_score(pred_df_swatp_arima_ssp126['obs'],pred_df_swatp_arima_ssp126['pred']),TCS(pred_df_swatp_arima_ssp126['obs'],pred_df_swatp_arima_ssp126['pred']))
    print('R2,TCS for SWATPlus+ARIMA SSP245:',r2_score(pred_df_swatp_arima_ssp245['obs'],pred_df_swatp_arima_ssp245['pred']),TCS(pred_df_swatp_arima_ssp245['obs'],pred_df_swatp_arima_ssp245['pred']))
    print('R2,TCS for SWATPlus+ARIMA SSP585:',r2_score(pred_df_swatp_arima_ssp585['obs'],pred_df_swatp_arima_ssp585['pred']),TCS(pred_df_swatp_arima_ssp585['obs'],pred_df_swatp_arima_ssp585['pred']))
    # visualize
    fig = px.line(pred_df_swatp_arima_simyr,x=pred_df_swatp_arima_simyr.index,y=['obs'],labels={'index':'Date','obs':'Observed'},title=f'Models of {hydro_station}')
    fig.add_trace(go.Scatter(x=SWATPlus_simu.index,y=SWATPlus_simu['Value'],mode='lines',name='SWATPlus Simulated',line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df_monthly_avg.index,y=df_monthly_avg['flow(m^3/s)'],mode='lines',name='Average Monthly Flow',line=dict(color='black')))
    fig.add_trace(go.Scatter(x=arima_pred.index,y=arima_pred['flow(m^3/s)'],mode='lines',name='SARIMA Pred',line=dict(color='tomato')))
    fig.add_trace(go.Scatter(x=pred_df_swatp_simyr.index,y=pred_df_swatp_simyr['pred'],mode='lines',name='SWATPlus Pred',line=dict(color='purple')))
    fig.add_trace(go.Scatter(x=pred_df_swatp_arima_simyr.index,y=pred_df_swatp_arima_simyr['pred'],mode='lines',name='SWATPlus-ARIMA Pred',line=dict(color='green')))
    fig.add_trace(go.Scatter(x=pred_df_swatp_arima_ssp126.index,y=pred_df_swatp_arima_ssp126['pred'],mode='lines',name='SWATPlus-ARIMA SSP126',line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=pred_df_swatp_arima_ssp245.index,y=pred_df_swatp_arima_ssp245['pred'],mode='lines',name='SWATPlus-ARIMA SSP245',line=dict(color='orange')))
    fig.add_trace(go.Scatter(x=pred_df_swatp_arima_ssp585.index,y=pred_df_swatp_arima_ssp585['pred'],mode='lines',name='SWATPlus-ARIMA SSP585',line=dict(color='red')))
    fig.add_trace(go.Scatter(x=pred_df_swatp_ssp126.index,y=pred_df_swatp_ssp126['pred'],mode='lines',name='SWATPlus SSP126',line=dict(color='lightblue')))
    fig.add_trace(go.Scatter(x=pred_df_swatp_ssp245.index,y=pred_df_swatp_ssp245['pred'],mode='lines',name='SWATPlus SSP245',line=dict(color='cyan')))
    fig.add_trace(go.Scatter(x=pred_df_swatp_ssp585.index,y=pred_df_swatp_ssp585['pred'],mode='lines',name='SWATPlus SSP585',line=dict(color='pink')))
    fig.show()


R2,TSC for average monthly flow: 0.5856823470085697 0.7446808510638298
R2,TCS for corrected SWATPlus-ARIMA: 0.7620963464484992 0.44680851063829785
R2,TCS for SARIMA: 0.83216858677616 0.7021276595744681
R2,TCS for SWATPlus SimYr: 0.5308962941607251 0.48936170212765956
R2,TCS for SWATPlus SSP126: 0.10577735041234937 0.14893617021276595
R2,TCS for SWATPlus SSP245: 0.1037784225691204 0.23404255319148937
R2,TCS for SWATPlus SSP585: 0.11912608929406865 0.14893617021276595
R2,TCS for SWATPlus+ARIMA SimYr: 0.7620963464484992 0.44680851063829785
R2,TCS for SWATPlus+ARIMA SSP126: 0.7288327411823297 0.19148936170212766
R2,TCS for SWATPlus+ARIMA SSP245: 0.7254826872641913 0.2765957446808511
R2,TCS for SWATPlus+ARIMA SSP585: 0.7255084116602744 0.19148936170212766


R2,TSC for average monthly flow: 0.598014058042422 0.8297872340425532
R2,TCS for corrected SWATPlus-ARIMA: 0.8160391359728333 0.23404255319148937
R2,TCS for SARIMA: 0.7898765715438034 0.3617021276595745
R2,TCS for SWATPlus SimYr: 0.5782503636440353 0.23404255319148937
R2,TCS for SWATPlus SSP126: -0.11398973826928338 0.06382978723404255
R2,TCS for SWATPlus SSP245: -0.12699092003252477 0.10638297872340426
R2,TCS for SWATPlus SSP585: -0.041770497392515704 0.10638297872340426
R2,TCS for SWATPlus+ARIMA SimYr: 0.8160391359728333 0.23404255319148937
R2,TCS for SWATPlus+ARIMA SSP126: 0.7978991448306356 0.06382978723404255
R2,TCS for SWATPlus+ARIMA SSP245: 0.7877147427900626 0.14893617021276595
R2,TCS for SWATPlus+ARIMA SSP585: 0.7326366890925943 0.10638297872340426


R2,TSC for average monthly flow: 0.6101806953852165 0.8297872340425532
R2,TCS for corrected SWATPlus-ARIMA: 0.7910651951318517 0.3617021276595745
R2,TCS for SARIMA: 0.8447823505748283 0.7021276595744681
R2,TCS for SWATPlus SimYr: 0.4886320816078288 0.3617021276595745
R2,TCS for SWATPlus SSP126: -0.1734746871639261 0.23404255319148937
R2,TCS for SWATPlus SSP245: -0.1750162534007469 0.2765957446808511
R2,TCS for SWATPlus SSP585: -0.11537961691288245 0.23404255319148937
R2,TCS for SWATPlus+ARIMA SimYr: 0.7910651951318517 0.3617021276595745
R2,TCS for SWATPlus+ARIMA SSP126: 0.7950411953614642 0.23404255319148937
R2,TCS for SWATPlus+ARIMA SSP245: 0.7893918797558773 0.2765957446808511
R2,TCS for SWATPlus+ARIMA SSP585: 0.7487832920979924 0.23404255319148937
